In [ ]:
interface ViewEvent
interface ViewState
interface ViewSideEffect

abstract class MVIViewModel<Event : ViewEvent, UiState : ViewState, Effect : ViewSideEffect> :
    ViewModel() {

    private val initialState: UiState by lazy { setInitialState() }
    abstract fun setInitialState(): UiState

    protected open suspend fun awaitReadiness(): Boolean = true

    protected open val readinessTimeoutMillis: Long = 2000L

    private val _viewState: MutableStateFlow<UiState> = MutableStateFlow(initialState)
    val viewState: StateFlow<UiState> = _viewState.onStart {
        val ready = awaitReadinessWithTimeout()
        if (ready) {
            onStart()
        } else {
            handleReadinessFailed()
        }
    }.stateIn(
        scope = viewModelScope,
        started = SharingStarted.WhileSubscribed(5000),
        initialValue = initialState
    )

    abstract fun onStart()

    private val _event: MutableSharedFlow<Event> = MutableSharedFlow()

    private val _effect: MutableSharedFlow<Effect> = MutableSharedFlow()
    val effect: SharedFlow<Effect> = _effect

    init {
        subscribeToEvents()
    }

    fun setEvent(event: Event) {
        viewModelScope.launch { _event.emit(event) }
    }

    protected fun setState(reducer: UiState.() -> UiState) {
        _viewState.update { currentState -> currentState.reducer() }
    }

    private fun subscribeToEvents() {
        viewModelScope.launch {
            _event.collect {
                handleEvents(it)
            }
        }
    }

    abstract fun handleEvents(event: Event)

    protected fun setEffect(builder: () -> Effect) {
        val effectValue = builder()
        viewModelScope.launch { _effect.emit(effectValue) }
    }

    /**
     * If awaitReadiness() returns false, handle that scenario here.
     * For example, show an error message or navigate away.
     */
    protected open fun handleReadinessFailed() {
        Logger.e("MVIViewModel: Readiness check failed")
        println("MVIViewModel: Readiness check failed")
        // By default, do nothing. Concrete ViewModels can override if needed.
        // Example:
        // setState { copy(errorMessage = "Initialization failed") }
        // setEffect { MyEffect.ShowError("Initialization failed") }
    }

    /**
     * Internal function to handle timeout in readiness checks.
     */
    private suspend fun awaitReadinessWithTimeout(): Boolean {
        return withTimeoutOrNull(readinessTimeoutMillis) {
            awaitReadiness()
        } ?: false
    }
}